In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3

In [ ]:
# Dimensions des images attendues par InceptionV3
img_height, img_width = 75, 75
num_classes = 7  # Nombre de classes dans FER2013

# Charger le modèle InceptionV3 sans les couches supérieures (top) et avec les poids de ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Dégeler les couches proches du fully connected
for layer in base_model.layers[-50:]:
    layer.trainable = True  # Dégeler les dernières 50 couches
# Ajouter des couches personnalisées pour la classification FER2013
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)  # Couche dense avec 256 neurones
x = Dropout(0.5)(x)  # Dropout pour éviter le surapprentissage
predictions = Dense(num_classes, activation='softmax')(x)  # Couche de sortie

# Définir le modèle complet
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 75, 75, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 37, 37, 32)     │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 37, 37, 32)     │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 37, 37, 32)     │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 35, 35, 32)     │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 35, 35, 32)     │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 35, 35, 32)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 35, 35, 64)     │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 35, 35, 64)     │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 35, 35, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 17, 17, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 17, 17, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 17, 17, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 17, 17, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 15, 15, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 15, 15, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 22,329,127 (85.18 MB)

 Trainable params: 22,294,695 (85.05 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [ ]:
Model: "functional_5"

In [ ]:
datagen_train = ImageDataGenerator(
    rescale=1./255,  # Normalisation des pixels
    rotation_range=30,  # Rotation des images aléatoire de 30 degrés
    width_shift_range=0.2,  # Déplacement horizontal aléatoire de 20%
    height_shift_range=0.2,  # Déplacement vertical aléatoire de 20%
    shear_range=0.2,  # Transformation de cisaillement (shearing)
    zoom_range=0.2,  # Zoom aléatoire des images
    horizontal_flip=True,  # Retourner horizontalement les images
    fill_mode='nearest',  # Mode de remplissage des pixels vides après transformation
    validation_split=0.2  # Diviser les données en ensemble d'entraînement et de validation
)

# Prétraitement pour les données de validation (sans augmentation, juste normalisation)
datagen_val = ImageDataGenerator(
    rescale=1./255,  # Normalisation des pixels
    validation_split=0.2  # Diviser les données en ensemble d'entraînement et de validation
)

# Générateur pour les images d'entraînement avec augmentation
training_set = datagen_train.flow_from_directory(
    '/content/drive/MyDrive/FER2013/fe2013/train',  # Répertoire des images d'entraînement
    target_size=(img_height, img_width),  # Redimensionner les images
    batch_size=32,
    class_mode='categorical',  # Type de classification (ici, multiclasse)
    subset='training'  # Ensemble d'entraînement
)
 #Générateur pour les images de validation sans augmentation
validation_set = datagen_val.flow_from_directory(
    '/content/drive/MyDrive/FER2013/fe2013/train',  # Répertoire des images de validation
    target_size=(img_height, img_width),  # Redimensionner les images
    batch_size=32,
    class_mode='categorical',  # Type de classification (ici, multiclasse)
    subset='validation'  # Ensemble de validation
)

Found 3113 images belonging to 7 classes.
Found 777 images belonging to 7 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


# EarlyStopping pour arrêter l'entraînement si la perte de validation ne s'améliore pas
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1)

# ModelCheckpoint pour sauvegarder le meilleur modèle
checkpoint = ModelCheckpoint(
    filepath='best_model.keras',  # Nom du fichier pour le modèle sauvegardé
    monitor='val_loss',
     save_best_only=True,      # Sauvegarder uniquement si c'est le meilleur modèle
    mode='min',               # Sauvegarder pour la perte minimale
    verbose=1
)

# Entraîner le modèle
history = model.fit(
    training_set,
    epochs=10,
    batch_size=32,
    validation_data=validation_set,
    shuffle=True,
    callbacks=[early_stop, checkpoint]  # Ajouter des callbacks pour early stopping et checkpoint
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.1482 - loss: 2.0858 
Epoch 1: val_loss improved from inf to 1.98906, saving model to best_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 2629s 25s/step - accuracy: 0.1482 - loss: 2.0854 - val_accuracy: 0.1557 - val_loss: 1.9891
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1669 - loss: 1.9787
Epoch 2: val_loss improved from 1.98906 to 1.91967, saving model to best_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.1669 - loss: 1.9787 - val_accuracy: 0.2059 - val_loss: 1.9197
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1757 - loss: 1.9588
Epoch 3: val_loss did not improve from 1.91967
98/98 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.1757 - loss: 1.9588 - val_accuracy: 0.2342 - val_loss: 1.9653
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2137 - loss: 1.9197
Epoch 4: val_loss improved from 1.91967 to 1.89794, saving model to best_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
test_set = datagen.flow_from_directory(
    '/content/drive/MyDrive/FER2013/fe2013/test',
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

from keras.models import load_model

# Charger le modèle sauvegardé
best_model = load_model('best_model.keras')


test_loss, test_accuracy = best_model.evaluate(test_set)
print(f'Test Accuracy: {test_accuracy:.2f}')

Found 2543 images belonging to 7 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


80/80 ━━━━━━━━━━━━━━━━━━━━ 952s 12s/step - accuracy: 0.3112 - loss: 1.7065
Test Accuracy: 0.32


In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Charger le modèle sauvegardé
model = load_model('best_model.keras')

# Dimensions des images attendues par le modèle
img_height, img_width = 75, 75
classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']  # Classes FER2013
def predict_emotion(image):
    # Redimensionner l'image à la taille requise
    image = image.resize((img_width, img_height))

    # Convertir l'image en tableau numpy et normaliser
    image_array = img_to_array(image) / 255.0  # Normalisation
    image_array = np.expand_dims(image_array, axis=0)  # Ajouter une dimension batch

    # Effectuer une prédiction
    predictions = model.predict(image_array)
    predicted_class = classes[np.argmax(predictions)]  # Classe avec la probabilité maximale
    confidence = np.max(predictions)  # Probabilité associée

    return f"Emotion: {predicted_class} (Confidence: {confidence:.2f})"
# Interface Gradio
interface = gr.Interface(
    fn=predict_emotion,  # Fonction de prédiction
    inputs=gr.Image(type="pil"),  # Entrée : une image (Pillow)
    outputs="text",  # Sortie : du texte (prédiction)
    title="FER2013 Emotion Recognition",
    description="Téléchargez une image faciale pour prédire l'émotion."
)

# Lancer l'application
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://17aff86f7887c8042e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
